In [1]:

import random
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras import backend as K

2023-03-20 21:41:18.122604: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 21:41:18.260179: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-20 21:41:18.263732: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-20 21:41:18.263751: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np

df = pd.read_csv("synthetic_encoded_dataset.csv")
df.shape

(79844, 14)

In [3]:
from sklearn.model_selection import train_test_split

y = df.y.to_frame()
X = df.drop(columns = ["y"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [4]:
ids = np.arange(0, df.shape[0])
df["ClientID"] = ids
df.head()
df.shape

(79844, 15)

In [5]:
def create_clients(image_list, label_list, num_clients=100, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = X
    data["y"] = y
    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))} 

## Distributing the dataset according to location among clients

In [6]:
clients = create_clients(X_train, y_train, num_clients=11, initial='client')
clients['client_1']

,age,education,default,balance,housing,loan,contact,campaign,pdays,previous,marital_type,job_type,poutcome_type,y
0,58,2,0,0.032119,1,0,0,1,999,0,1,4,3,0
1,44,1,0,0.000435,1,0,0,1,999,0,2,9,3,0
2,33,1,0,0.000030,1,1,0,1,999,0,1,2,3,0
3,47,1,0,0.022572,1,0,0,1,999,0,1,1,3,0
4,33,1,0,0.000015,0,0,0,1,999,0,2,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7253,48,0,0,0.008258,1,0,0,2,999,0,1,10,3,0
7254,39,1,1,-0.000240,0,0,0,2,999,0,1,1,3,0
7255,31,2,0,0.000120,0,0,0,5,999,0,1,9,3,0
7256,35,1,0,0.013759,1,0,0,1,999,0,2,7,3,0


In [7]:
def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    label = data_shard.y.to_frame()
    data = data_shard.drop(columns = ["y"])
    # print((data,label))
    dataset = tf.data.Dataset.from_tensor_slices((data, label))
    # print(data.shape)
    # print(dataset.shape)
    # for i in dataset:
    #     print(i)
    #     break
    return dataset.shuffle(len(label)).batch(bs)

In [8]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
  # print(list(data))
  clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

2023-03-20 21:41:22.119654: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-20 21:41:22.119690: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-20 21:41:22.119704: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dhiraj-Inspiron-5593): /proc/driver/nvidia/version does not exist
2023-03-20 21:41:22.119970: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Batching the data
The data is further batched for the purpose of training. It is also converted into tensors so that keras can be applied on it.

In [9]:
for x, y in clients_batched['client_1']:
  print(x,y)

tf.Tensor(
[[ 5.30000000e+01  1.00000000e+00  0.00000000e+00  1.61103700e-01
   1.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   9.99000000e+02  0.00000000e+00  1.00000000e+00  7.00000000e+00
   3.00000000e+00]
 [ 3.90000000e+01  1.00000000e+00  0.00000000e+00  2.69780129e-04
   1.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   9.99000000e+02  0.00000000e+00  1.00000000e+00  4.00000000e+00
   3.00000000e+00]
 [ 4.00000000e+01  2.00000000e+00  0.00000000e+00  7.34401463e-03
   1.00000000e+00  0.00000000e+00  0.00000000e+00  2.00000000e+00
   9.99000000e+02  0.00000000e+00  0.00000000e+00  4.00000000e+00
   3.00000000e+00]
 [ 5.70000000e+01  1.00000000e+00  0.00000000e+00  2.42802116e-03
   1.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   9.99000000e+02  0.00000000e+00  1.00000000e+00  7.00000000e+00
   3.00000000e+00]
 [ 3.10000000e+01  1.00000000e+00  0.00000000e+00  4.64621334e-04
   1.00000000e+00  1.00000000e+00  0.00000000e+00  2.00

In [10]:
class SimpleMLP:
  @staticmethod
  def build(shape):
    model = Sequential()
    model.add(Dense(16, input_shape=(shape,), activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model
    return model

In [11]:
lr = 0.01 
comms_round = 100
loss= tensorflow.keras.losses.BinaryCrossentropy()
metrics = ['accuracy']
optimizer = SGD(learning_rate=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               )

In [12]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    logits = logits.astype('int64')
    acc = accuracy_score(logits, Y_test)
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [13]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(13)

print(comms_round)
        
# commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(13)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

100
250/250 [==============================] - 0s 1ms/step
comm_round: 0 | global_acc: 49.430% | global_loss: 0.7248246073722839
250/250 [==============================] - 0s 2ms/step
comm_round: 1 | global_acc: 49.430% | global_loss: 0.7238082885742188
250/250 [==============================] - 1s 2ms/step
comm_round: 2 | global_acc: 49.430% | global_loss: 0.7231503129005432
250/250 [==============================] - 0s 2ms/step
comm_round: 3 | global_acc: 49.430% | global_loss: 0.7225712537765503
250/250 [==============================] - 0s 2ms/step
comm_round: 4 | global_acc: 49.430% | global_loss: 0.7223206162452698
250/250 [==============================] - 1s 2ms/step
comm_round: 5 | global_acc: 49.430% | global_loss: 0.7226753830909729
250/250 [==============================] - 0s 2ms/step
comm_round: 6 | global_acc: 49.430% | global_loss: 0.7226311564445496
250/250 [==============================] - 0s 1ms/step
comm_round: 7 | global_acc: 49.430% | global_loss: 0.7220409512519

In [14]:

def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    logits = logits.astype('int64')
    acc = accuracy_score(logits,Y_test)
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [15]:
SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(y_train)).batch(320)
smlp_SGD = SimpleMLP()
SGD_model = smlp_SGD.build(13) 

SGD_model.compile(loss=loss, 
              optimizer=optimizer, 
              metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

#test the SGD global model and print out metrics
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)

250/250 [==============================] - 0s 940us/step
comm_round: 1 | global_acc: 49.430% | global_loss: 0.7211562991142273
